In [51]:
from config import API_KEY
import requests
import sample
import pandas as pd


In [52]:
API_HOST = 'https://api.yelp.com'
SEARCH_PATH = '/v3/businesses/search'
BUSINESS_PATH = '/v3/businesses'

file_name_city = 'city_list.csv'
final_file = 'gym_concept_ratio.csv'
CITY_STATE = pd.read_csv(file_name_city)
CITY_STATE['location'] = CITY_STATE.city + ',' + CITY_STATE.state

# CITY = ['Atlanta', 'Chicago', 'Dallas', 'Denver', 'Detroit', 'Los Angeles','Miami', 'Philadelphia', 'New York']
# STATE = ['Georgia', 'Illinois', 'Texas', 'Colorado', 'Michigan', 'California','Florida', 'Pennsylvania', 'New York']

In [53]:
locations = []

for location in CITY_STATE['location']:
    locations.append(location)
locations

['Atlanta,GA',
 'Chicago,IL',
 'Dallas,TX',
 'Denver,CO',
 'Detroit,MI',
 'Los Angeles,CA',
 'Miami,FL',
 'Philadelphia,PA',
 'New York,NY',
 'Seattle,WA',
 'Columbus,OH',
 'Nashville,TN',
 'Boston,MA',
 'Washington,DC']

In [54]:
#For test: locations = ["Austin,TX", "Round Rock,TX"]
term = "gyms"
limit = 50
open_at_time = 1528961400 # Gyms open on Thursday 1:30 a.m. CST

In [55]:
# get total gyms

def get_total_gym(term, locations):
    
    total_gym = []
    
    for location in locations:
        params={
            "term" : term,
            "location" : location,
            "limit" : 50,
        }
        
        response = sample.request(API_HOST, SEARCH_PATH, API_KEY, params)
        total = response['total']
        print(f"get_total_gym: {total} gyms in {location}")
        
        total_gym.append(total)
        
    final_df = pd.DataFrame({
         "location" : locations,
        "total_gyms" : total_gym  
    })
        
    return final_df

In [56]:
test_df = get_total_gym(term,locations)

Querying https://api.yelp.com/v3/businesses/search ...
get_total_gym: 1500 gyms in Atlanta,GA
Querying https://api.yelp.com/v3/businesses/search ...
get_total_gym: 2300 gyms in Chicago,IL
Querying https://api.yelp.com/v3/businesses/search ...
get_total_gym: 1600 gyms in Dallas,TX
Querying https://api.yelp.com/v3/businesses/search ...
get_total_gym: 1600 gyms in Denver,CO
Querying https://api.yelp.com/v3/businesses/search ...
get_total_gym: 718 gyms in Detroit,MI
Querying https://api.yelp.com/v3/businesses/search ...
get_total_gym: 6600 gyms in Los Angeles,CA
Querying https://api.yelp.com/v3/businesses/search ...
get_total_gym: 1300 gyms in Miami,FL
Querying https://api.yelp.com/v3/businesses/search ...
get_total_gym: 1400 gyms in Philadelphia,PA
Querying https://api.yelp.com/v3/businesses/search ...
get_total_gym: 5100 gyms in New York,NY
Querying https://api.yelp.com/v3/businesses/search ...
get_total_gym: 1600 gyms in Seattle,WA
Querying https://api.yelp.com/v3/businesses/search ...


In [57]:
def get_total_24hr_gym(term, locations, open_at_time):
    
    total_gym = []
    
    for location in locations:
        params={
            "term" : term,
            "location" : location,
            "open_at" : open_at_time,
            "limit" : 50,
        }
        
        response = sample.request(API_HOST, SEARCH_PATH, API_KEY, params)
        total = response['total']
        print(f"get_total_24hr_gym: {total} gyms in {location}")
        
        total_gym.append(total)
        
    final_df = pd.DataFrame({
         "location" : locations,
        "total_24hr_gyms" : total_gym  
    })
        
    return final_df

In [58]:
def main(term, locations, open_at_time, file_name):
    total_24hr_gym = get_total_24hr_gym(term, locations, open_at_time)
    total_gym = get_total_gym(term, locations)
    combined_gym = pd.merge(total_gym, total_24hr_gym, on='location', left_index=True)
    combined_gym['ratio%'] = (combined_gym.total_24hr_gyms/combined_gym.total_gyms*100).round(2)
    combined_gym.to_csv(file_name, index=False)
    return combined_gym

In [59]:
final = main(term, locations, open_at_time, 'gym_ratio.csv')

Querying https://api.yelp.com/v3/businesses/search ...
get_total_24hr_gym: 115 gyms in Atlanta,GA
Querying https://api.yelp.com/v3/businesses/search ...
get_total_24hr_gym: 114 gyms in Chicago,IL
Querying https://api.yelp.com/v3/businesses/search ...
get_total_24hr_gym: 128 gyms in Dallas,TX
Querying https://api.yelp.com/v3/businesses/search ...
get_total_24hr_gym: 96 gyms in Denver,CO
Querying https://api.yelp.com/v3/businesses/search ...
get_total_24hr_gym: 69 gyms in Detroit,MI
Querying https://api.yelp.com/v3/businesses/search ...
get_total_24hr_gym: 185 gyms in Los Angeles,CA
Querying https://api.yelp.com/v3/businesses/search ...
get_total_24hr_gym: 47 gyms in Miami,FL
Querying https://api.yelp.com/v3/businesses/search ...
get_total_24hr_gym: 86 gyms in Philadelphia,PA
Querying https://api.yelp.com/v3/businesses/search ...
get_total_24hr_gym: 152 gyms in New York,NY
Querying https://api.yelp.com/v3/businesses/search ...
get_total_24hr_gym: 44 gyms in Seattle,WA
Querying https://ap

In [60]:
final["city"] = final["location"].apply(lambda x: x.split(",")[0])
final["state"] = final["location"].apply(lambda x: x.split(",")[1])
final

,location,total_gyms,total_24hr_gyms,ratio%,city,state
0,"Atlanta,GA",1500,115,7.67,Atlanta,GA
1,"Chicago,IL",2300,114,4.96,Chicago,IL
2,"Dallas,TX",1600,128,8.00,Dallas,TX
3,"Denver,CO",1600,96,6.00,Denver,CO
4,"Detroit,MI",718,69,9.61,Detroit,MI
5,"Los Angeles,CA",6600,185,2.80,Los Angeles,CA
6,"Miami,FL",1300,47,3.62,Miami,FL
7,"Philadelphia,PA",1400,86,6.14,Philadelphia,PA
8,"New York,NY",5100,152,2.98,New York,NY
9,"Seattle,WA",1600,44,2.75,Seattle,WA


In [61]:
populations = pd.read_csv("city_list.csv")

In [62]:
populations

,city,state,population
0,Atlanta,GA,"491,626"
1,Chicago,IL,2687682
2,Dallas,TX,1359133
3,Denver,CO,719116
4,Detroit,MI,665713
5,Los Angeles,CA,4030668
6,Miami,FL,479009
7,Philadelphia,PA,1573688
8,New York,NY,8580015
9,Seattle,WA,746046


In [63]:
final = pd.merge(final, populations, on= ["city", "state"], how="inner")